In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    #filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, normaltest

In [ ]:
#Wczytanie bazy danych do lokalnej zmiennej
data = pd.read_csv("../input/videogamesales/vgsales.csv")
data.head()

In [ ]:
data.info()

In [ ]:
data.isna().sum()

Statystyki dotyczące każdej z dostępnych zmiennych ilościowych

In [ ]:
quant_col = ['NA_Sales', 'EU_Sales', 'JP_Sales','Other_Sales', 'Global_Sales']
quant_stats = data[quant_col].agg(["count","mean","median","min", "max", "std", "var",])
quant_stats = quant_stats.append(data[quant_col].mode().rename(index={0:"mode"}))
quant_stats

Tabele liczności dla zmiennych jakościowych 

In [ ]:
quali_cols = ['Name', 'Platform','Year', 'Genre']
fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quali_cols):
    axes[i].set_title(f"Wykres rozkładu liczności dla {col}")
    sns.histplot(data[col], ax=axes[i])
    print(f"""Dla kolumny {col} tabela liczności wygląda następująco:\n\n\n{pd.DataFrame(data[col].value_counts())}\n""")

Z powyższych histogramów można wywnioskować, że najwięcej gier sprzedano w latach 2008 oraz 2009 gdzie najpopularniejszą platformą docelową gry było DS oraz PS2. Dominują gry akcji.

Histogram analizy kolumn

In [ ]:
plotPerColumnDistribution(data, 10, 5)

Macierz korelacji

In [ ]:
plotCorrelationMatrix(data, 8)

Wykres rozproszenia i gęstości

In [ ]:
plotScatterMatrix(data, 18, 10)

Uzupełnienie kolumny "Year" wartościami tak, aby pozbyć się nulli. Nie zdecydowałem się ich usunąć, lecz uzupełnić średnią wartością. Dzięki temu wiemy, że najwięcej gier spprzedano w 2008 oraz 2009 roku. 

In [ ]:
mean = data['Year'].mean()
data['Year'].fillna(mean, inplace=True)
plt.xticks(rotation = 75)
x_axis = data['Year'].astype(int)
sns.countplot(x= x_axis, data = data)
plt.title('Total Game Sales Each Year')
plt.show()

In [ ]:
plt.xticks(rotation = 75)
x_axis = data['Genre']
sns.countplot(x= x_axis, data = data)
plt.title('Total Game Sales of Each Genre')
plt.show()

A najczęściej sprzedającym się typem gier były gry akcji. 

W każdym typie gier, najlepiej sprzedaje się 

In [ ]:
game_sales = data[['Genre','Name']].groupby(['Genre']).agg(lambda x:x.value_counts().index[0])
genre_games = game_sales.rename(columns = {'Name' : 'Game'}, inplace = False)
genre_games

Gry najlepiej sprzedawały się na platformę PS2 oraz DS

In [ ]:
plt.xticks(rotation = 75)
x_axis = data['Platform']
sns.countplot(x= x_axis, data = data)
plt.title('Total Game Sales on Each Platform')
plt.show()

Macierz korelacji, uwzględniająca także rok powstania gry

In [ ]:
correlation_vg_sales = data.corr()

axis_corr = sns.heatmap(
correlation_vg_sales,
vmin=-1, vmax=1, center=0,
cmap=sns.diverging_palette(50, 500, n=500),
square=True
)

plt.show()

Dla pustych pól wydawcy gier "Publisher" uzupełnione zostały one wartościami "Unknown"

In [ ]:
data['Publisher'] = data['Publisher'].fillna('Unknown')
number_df = data.groupby('Publisher')[['Name']].count().sort_values('Name', ascending = False).head(50)
number_clean = number_df.rename(columns = {'Name' : 'Number'}, inplace = False)
number_clean

Gdzie najpopularniejszym producentem gier był zdecydowanie "Eloctronic Arts"

In [ ]:
number_df.plot(kind = 'bar', figsize = (25, 10));
plt.xlabel('Publisher', fontsize = 20);
plt.ylabel('Number of video games released', fontsize = 20);
plt.title('Top Publishers of Games', fontsize = 40)

W każdym regionie świata podział sprzedaży gier ma się następująco: 

Północna Ameryka:

In [ ]:
top_games_NA = data.sort_values('NA_Sales',ascending = False).head(5)
explode = [0.1, 0, 0, 0, 0]
plt.pie(top_games_NA['NA_Sales'], labels = top_games_NA['Name'], explode = explode)
plt.show()

Europa

In [ ]:
top_games_EU = data.sort_values('EU_Sales',ascending = False).head(5)
explode = [0.1, 0, 0, 0, 0]
plt.pie(top_games_EU['EU_Sales'], labels = top_games_EU['Name'], explode = explode)
plt.show()

Japonia:

In [ ]:
top_games_JP = data.sort_values('JP_Sales',ascending = False).head(5)
explode = [0.1, 0, 0, 0, 0]
plt.pie(top_games_JP['JP_Sales'], labels = top_games_JP['Name'], explode = explode)
plt.show()

Pozostałe:

In [ ]:
top_games_Other = data.sort_values('Other_Sales',ascending = False).head(5)
explode = [0.1, 0, 0, 0, 0]
plt.pie(top_games_Other['Other_Sales'], labels = top_games_Other['Name'], explode = explode)
plt.show()

Globalna sprzedaż

In [ ]:
top_games_Global = data.sort_values('Global_Sales',ascending = False).head(5)
explode = [0.1, 0, 0, 0, 0]
plt.pie(top_games_Global['Global_Sales'], labels = top_games_Global['Name'], explode = explode)
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
plt.scatter(data.Genre,data.NA_Sales,color='b',alpha=.5)
plt.xlabel('Typ gier')           
plt.ylabel('Ilość')
plt.title('Rozkład ')           
plt.show()

Procentowy wykres sprzedaży na regiony świata

In [ ]:
total_sales = pd.DataFrame({'Country':['NA', 'EU', 'JP', 'Other'], 'Sales':[sum(data['NA_Sales']), sum(data['EU_Sales']), sum(data['JP_Sales']), sum(data['Other_Sales'])]})
total_sales['Percentages'] = total_sales['Sales']/sum(total_sales['Sales'])*100
total_sales.sort_values(by='Percentages', inplace=True)

In [ ]:
fig = plt.figure(facecolor='whitesmoke')

axes1 = fig.add_axes([0, 0, 1, 1])
axes2 = fig.add_axes([0.7, 0, 1, 1])
axes3 = fig.add_axes([1.4, 0, 1, 1])
axes4 = fig.add_axes([2.1, 0, 1, 1])

axes1.pie([total_sales['Percentages'][0], 100-total_sales['Percentages'][0]], startangle=180,
         colors=['crimson', 'white'])
axes1.text(-0.27, -0.85,f"{round(total_sales['Percentages'][0],2)}%", fontweight='bold', fontsize=16)
axes1.text(-0.5, 1.3, 'North America', fontweight='bold', fontsize=16)

axes2.pie([total_sales['Percentages'][1], 100-total_sales['Percentages'][1]], startangle=-4,
         colors=['crimson', 'white'])
axes2.text(0.26, 0.355, f"{round(total_sales['Percentages'][1],2)}%", fontweight='bold', fontsize=16)
axes2.text(-0.3, 1.3, 'Europe', fontweight='bold', fontsize=16)

axes3.pie([total_sales['Percentages'][2], 100-total_sales['Percentages'][2]], startangle=95,
         colors=['crimson', 'white'])
axes3.text(-0.62, 0.5, f"{round(total_sales['Percentages'][2],2)}%", fontweight='bold', fontsize=16)
axes3.text(-0.25, 1.3, 'Japan', fontweight='bold', fontsize=16)

axes4.pie([total_sales['Percentages'][3], 100-total_sales['Percentages'][3]], startangle=130,
         colors=['crimson', 'white'])
axes4.text(-0.77, 0.3, f"{round(total_sales['Percentages'][3],2)}%", fontweight='bold', fontsize=16)
axes4.text(-0.5, 1.3, 'Other countries', fontweight='bold', fontsize=16)

axes2.text(-0.3,1.8, 'Percentage of sales by country', fontweight='bold', color='crimson', fontsize=24)
fig.show()

In [ ]:
sales_by_year = data.groupby('Year')[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']].sum().reset_index()

fig = plt.figure(facecolor='whitesmoke', figsize=(6,5))
sns.set_style('white')
axes1 = fig.add_axes([0, 0, 1.5, 1.5]) 
axes2 = fig.add_axes([1.6, 0, 1, 1.5]) 
axes3 = fig.add_axes([0, -1.8, 1.5, 1.5])
axes4 = fig.add_axes([1.6, -1.8, 1, 1.5]) 

# figure
sns.lineplot(x=sales_by_year['Year'], y=sales_by_year['NA_Sales'],color='crimson',lw=4, ax=axes1)
axes1.scatter(x=[2000, 2003, 2012, 2013, 2008], y=[94.49, 193.59, 154.93, 154.77, 351.44], color='black', lw=4)
axes1.set_facecolor('whitesmoke')
axes1.set_xlabel('Year', fontsize=18, color='black')
axes1.set_ylabel('Sales', fontsize=18, color='black')
axes1.text(1979, 340, 'Sales in', color='black', fontsize=20, fontweight='bold')
axes1.text(1985, 340, 'North America', color='crimson', fontsize=24, fontweight='bold')

# growth
axes1.annotate('', xy=(2006.9, 350), xytext=(1993.6, 60),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('Strong growth with 2 downturns in', xy=(1993.6, 100), xytext=(1993.6, 100), rotation=61.5, color='black', fontweight='bold', fontsize=12)
axes1.annotate('2000', xy=(2000.6, 250), xytext=(2000.6, 250), rotation=61.5, color='crimson', fontweight='bold', fontsize=12)
axes1.annotate('and', xy=(2001.8, 276), xytext=(2001.8, 276), rotation=61.5, color='black', fontweight='bold', fontsize=12)
axes1.annotate('2002', xy=(2002.8, 298), xytext=(2002.8, 298), rotation=61.5, color='crimson', fontweight='bold', fontsize=12)


# stagnation
axes1.annotate('', xy=(2012, 149), xytext=(2008, 100),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('', xy=(2013, 149), xytext=(2012, 100),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('Stagnation in', xy=(2000.5, 90), xytext=(2000.5, 90), color='black', fontweight='bold', fontsize=12)
axes1.annotate('2012', xy=(2006.3, 90), xytext=(2006.3, 90),color='crimson', fontweight='bold', fontsize=12)
axes1.annotate('and', xy=(2008.5, 90), xytext=(2008.5, 90),color='black', fontweight='bold', fontsize=12)
axes1.annotate('2013', xy=(2010.3, 90), xytext=(2010.3, 90),color='crimson', fontweight='bold', fontsize=12)

# max
axes1.annotate('', xy=(2009, 353.8), xytext=(2015, 325),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.3'))
axes1.annotate('Maximum sales', xy=(2009, 353.8), xytext=(2011, 317.5), color='black', fontweight='bold', fontsize=12)
axes1.annotate('in', xy=(2011, 303.5), xytext=(2011, 303.5), color='black', fontweight='bold', fontsize=12)
axes1.annotate('2008', xy=(2012, 303.5), xytext=(2012, 303.5), color='crimson', fontweight='bold', fontsize=12)

# conclusion № 1
axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.8, 'On the chart we can see that', color='black', fontsize=14)
axes2.text(0.48, 0.8, 'sales before 1996', color='crimson', fontsize=14)
axes2.text(0, 0.75, 'were highly volatile -', color='black', fontsize=14)
axes2.text(0.35, 0.75, 'growth was followed by donwturn.', color='crimson', fontsize=14)
axes2.text(0, 0.7, 'After 1996', color='crimson', fontsize=14)
axes2.text(0.18, 0.7, 'we observe', color='black', fontsize=14)
axes2.text(0.375, 0.7, 'strong growth', color='crimson', fontsize=14)
axes2.text(0.6, 0.7, ', but were two ', color='black', fontsize=14)
axes2.text(0, 0.65, 'downturns in 2000 and 2002.', color='crimson', fontsize=14)
axes2.text(0, 0.6, 'It should be noted that', color='black', fontsize=14)
axes2.text(0.383, 0.6, 'in 2008', color='crimson', fontsize=14)
axes2.text(0.518, 0.6, 'were the most sales -', color='black', fontsize=14)
axes2.text(0, 0.55, '351.44 millions.', color='crimson', fontsize=14)
axes2.text(0, 0.5, 'After 2008', color='crimson', fontsize=14)
axes2.text(0.185, 0.5, 'we can see a', color='black', fontsize=14)
axes2.text(0.41, 0.5, 'strong decrease in sales.', color='crimson', fontsize=14)
axes2.text(0, 0.45, 'In the period', color='black', fontsize=14)
axes2.text(0.22, 0.45, '2012 - 2013', color='crimson', fontsize=14)
axes2.text(0.43, 0.45, 'was', color='black', fontsize=14)
axes2.text(0.51, 0.45, 'stagnation.', color='crimson', fontsize=14)

# calculation of sales growth rates
list_NA = []
for n in range(1, 37):
    d = ((sales_by_year['NA_Sales'][n] - sales_by_year['NA_Sales'][n-1])/sales_by_year['NA_Sales'][n-1])*100
    list_NA.append(d)

# visualisation growth rates sales in NA by years
colors = ['crimson' if _ > 0 else 'darksalmon' for _ in list_NA]
axes3.bar(height=list_NA, x=sales_by_year['Year'][range(1, 37)], color=colors)
for p in axes3.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    if height >= 0:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.02), ha='center')
    elif height <= -55:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.2), ha='center')
    else:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.4), ha='center')
axes3.set_facecolor('whitesmoke')
axes3.set_xlabel('Year', fontsize=18, color='black')
axes3.set_ylabel('Growth rate, %', fontsize=18, color='black')
axes3.text(1997, 315, 'Growth rate sales(%) in', color='black', fontsize=20, fontweight='bold')
axes3.text(2001.39, 280, 'North America', color='crimson', fontsize=24, fontweight='bold')

# max +
axes3.annotate('', xy=(1985, 330), xytext=(1992, 285),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.3'))
axes3.annotate('Max', xy=(1990, 268), xytext=(1990, 268), color='black', fontweight='bold', fontsize=12)
axes3.annotate('+', xy=(1991.8, 268), xytext=(1991.8, 268), color='crimson', fontweight='bold', fontsize=24)

# max -
axes3.annotate('', xy=(2015, -79), xytext=(2008, -63),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.3'))
axes3.annotate('Max', xy=(2006, -46), xytext=(2006, -46), color='black', fontweight='bold', fontsize=12)
axes3.annotate('-', xy=(2007.8, -46), xytext=(2007.8, -46), color='darksalmon', fontweight='bold', fontsize=24)

# conclusion № 2
axes4.set_facecolor('whitesmoke')
axes4.axis('off')
axes4.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes4.text(0, 0.8, 'We can see that', color='black', fontsize=14)
axes4.text(0.27, 0.8, 'maximum positive growth rate', color='crimson', fontsize=14)
axes4.text(0, 0.75, 'sales in NA was', color='black', fontsize=14)
axes4.text(0.27, 0.75, 'in 1983 - 1984', color='crimson', fontsize=14)
axes4.text(0.52, 0.75, 'and', color='black', fontsize=14)
axes4.text(0.6, 0.75, 'maximum negative', color='crimson', fontsize=14)
axes4.text(0, 0.70, 'growth rate', color='crimson', fontsize=14)
axes4.text(0.21, 0.70, 'sales in NA was', color='black', fontsize=14)
axes4.text(0.48, 0.70, 'in 2015 - 2016.', color='crimson', fontsize=14)
axes4.text(0, 0.65, 'Average growth rates by period:', color='crimson', fontsize=14)
axes4.text(0, 0.60, f'* 1980-1984 - {round(statistics.mean(list_NA[:5]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.55, f'* 1985-1989 - {round(statistics.mean(list_NA[5:10]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.50, f'* 1990-1994 - {round(statistics.mean(list_NA[10:15]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.45, f'* 1995-1999 - {round(statistics.mean(list_NA[15:20]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.40, f'* 2000-2004 - {round(statistics.mean(list_NA[20:25]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.35, f'* 2005-2009 - {round(statistics.mean(list_NA[25:30]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.30, f'* 2010-2016 - {round(statistics.mean(list_NA[30:]),2)}%.', color='black', fontsize=14)
fig.show()

In [ ]:
fig = plt.figure(facecolor='whitesmoke')
sns.set_style('white')
axes1 = fig.add_axes([0, 0, 1.5, 1.5]) 
axes2 = fig.add_axes([1.6, 0, 1, 1.5]) 
axes3 = fig.add_axes([0, -1.8, 1.5, 1.5]) 
axes4 = fig.add_axes([1.6, -1.8, 1, 1.5]) 

sns.lineplot(x=sales_by_year['Year'], y=sales_by_year['EU_Sales'],color='crimson',lw=4, ax=axes1)
axes1.scatter(x=[2000, 2003, 2009, 2013, 2014], y=[52.75, 103.81, 191.59, 125.80, 125.63], color='black', lw=4)
axes1.set_facecolor('whitesmoke')
axes1.set_xlabel('Year', color='black', fontsize=18)
axes1.set_ylabel('Sales', color='black', fontsize=18)
axes1.text(1979, 180, 'Sales in', color='black', fontsize=20, fontweight='bold')
axes1.text(1985, 180, 'Europe', color='crimson', fontsize=24, fontweight='bold')

# growth
axes1.annotate('', xy=(2007, 193), xytext=(1993, 23),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('Strong growth with 2 downturns in', xy=(1992.4, 40), xytext=(1992.4, 40), rotation=56, color='black', fontweight='bold', fontsize=12)
axes1.annotate('2000', xy=(2000.7, 139.1), xytext=(2000.7, 139.1), rotation=56, color='crimson', fontweight='bold', fontsize=12)
axes1.annotate('and', xy=(2001.9, 154.2), xytext=(2001.9, 154.2), rotation=56, color='black', fontweight='bold', fontsize=12)
axes1.annotate('2003', xy=(2003, 166.7), xytext=(2003, 166.7), rotation=56, color='crimson', fontweight='bold', fontsize=12)

# stagnation
axes1.annotate('', xy=(2012.4, 116), xytext=(2008, 71),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('', xy=(2013.4, 116), xytext=(2012, 71),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('Stagnation in', xy=(2000.5, 61), xytext=(2000.5, 61), color='black', fontweight='bold', fontsize=12)
axes1.annotate('2013', xy=(2006.3, 61), xytext=(2006.3, 61),color='crimson', fontweight='bold', fontsize=12)
axes1.annotate('and', xy=(2008.5, 61), xytext=(2008.5, 61),color='black', fontweight='bold', fontsize=12)
axes1.annotate('2014', xy=(2010.3, 61), xytext=(2010.3, 61),color='crimson', fontweight='bold', fontsize=12)

# max
axes1.annotate('', xy=(2010, 194), xytext=(2016, 169.2),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.3'))
axes1.annotate('Maximum sales', xy=(2011.5, 161.7), xytext=(2011.5, 161.7), color='black', fontweight='bold', fontsize=12)
axes1.annotate('in', xy=(2011.5, 152), xytext=(2011.5, 152), color='black', fontweight='bold', fontsize=12)
axes1.annotate('2009', xy=(2012.5, 152), xytext=(2012.5, 152), color='crimson', fontweight='bold', fontsize=12)

# conclusion № 1
axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.8, 'On the chart we can see', color='black', fontsize=14)
axes2.text(0.41, 0.8, 'strong growth after 1994', color='crimson', fontsize=14)
axes2.text(0.825, 0.8, 'and two', color='black', fontsize=14)
axes2.text(0, 0.75, 'downturns in 2000 and 2003.', color='crimson', fontsize=14)
axes2.text(0, 0.7, 'And', color='black', fontsize=14)
axes2.text(0.07, 0.7, 'max sales', color='crimson', fontsize=14)
axes2.text(0.245, 0.7, 'were', color='black', fontsize=14)
axes2.text(0.333, 0.7, 'in 2009.', color='crimson', fontsize=14)
axes2.text(0.472, 0.7, 'We observe', color='black', fontsize=14)
axes2.text(0.675, 0.7, 'decrease sales', color='crimson', fontsize=14)
axes2.text(0, 0.65, 'after 2009', color='crimson', fontsize=14)
axes2.text(0.18, 0.65, 'with a', color='black', fontsize=14)
axes2.text(0.29, 0.65, 'small increase in 2011-2012.', color='crimson', fontsize=14)
axes2.text(0, 0.6, 'In period', color='black', fontsize=14)
axes2.text(0.155, 0.6, '2013 - 2014 sales almost unchanged.', color='crimson', fontsize=14)
axes2.text(0, 0.55, '----------------------------------------------------------------------------------------', color='black', fontsize=14)
axes2.text(0, 0.50, 'In comprasion with NA', color='crimson', fontsize=14)
axes2.text(0.375, 0.50, 'we can say that the', color='black', fontsize=14)
axes2.text(0.71, 0.50, 'situation', color='crimson', fontsize=14)
axes2.text(0, 0.45, 'is almost', color='black', fontsize=14)
axes2.text(0.16, 0.45, 'identical.', color='crimson', fontsize=14)
axes2.text(0, 0.4, 'Interestingly that', color='black', fontsize=14)
axes2.text(0.3, 0.4, 'maximum sales, decrease and stagnation', color='crimson', fontsize=14)
axes2.text(0, 0.35, 'in EU', color='crimson', fontsize=14)
axes2.text(0, 0.35, 'in EU', color='crimson', fontsize=14)
axes2.text(0.1, 0.35, 'were a', color='black', fontsize=14)
axes2.text(0.22, 0.35, 'year later', color='crimson', fontsize=14)
axes2.text(0.39, 0.35, 'tnan', color='black', fontsize=14)
axes2.text(0.47, 0.35, 'in NA.', color='crimson', fontsize=14)
fig.show()

# calculation of sales growth rates
list_EU = []
for n in range(1, 37):
    d = ((sales_by_year['EU_Sales'][n] - sales_by_year['EU_Sales'][n-1])/sales_by_year['EU_Sales'][n-1])*100
    list_EU.append(d)

# visualisation growth rates sales in EU by years
colors = ['crimson' if _ > 0 else 'darksalmon' for _ in list_EU]
axes3.bar(height=list_EU, x=sales_by_year['Year'][range(1, 37)], color=colors)
for p in axes3.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    if height >= 0:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.02), ha='center')
    elif height <= -55:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.2), ha='center')
    else:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.4), ha='center')
axes3.set_facecolor('whitesmoke')
axes3.set_xlabel('Year', fontsize=18, color='black')
axes3.set_ylabel('Growth rate, %', fontsize=18, color='black')
axes3.text(1997, 315, 'Growth rate sales(%) in', color='black', fontsize=20, fontweight='bold')
axes3.text(2007.48, 280, 'Europe', color='crimson', fontsize=24, fontweight='bold')

# max +
axes3.annotate('', xy=(1989, 368), xytext=(1996, 353),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.3'))
axes3.annotate('Max', xy=(1994, 336), xytext=(1994, 336), color='black', fontweight='bold', fontsize=12)
axes3.annotate('+', xy=(1995.8, 336), xytext=(1995.8, 336), color='crimson', fontweight='bold', fontsize=24)

# max -
axes3.annotate('', xy=(2015, -73), xytext=(2008, -57),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.3'))
axes3.annotate('Max', xy=(2006, -40), xytext=(2006, -40), color='black', fontweight='bold', fontsize=12)
axes3.annotate('-', xy=(2007.8, -40), xytext=(2007.8, -40), color='darksalmon', fontweight='bold', fontsize=24)

# conclusion № 2
axes4.set_facecolor('whitesmoke')
axes4.axis('off')
axes4.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes4.text(0, 0.8, 'From chart we can see that', color='black', fontsize=14)
axes4.text(0.455, 0.8, 'maximum positive growth', color='crimson', fontsize=14)
axes4.text(0, 0.75, 'was', color='black', fontsize=14)
axes4.text(0.07, 0.75, 'in 1987 - 1988,', color='crimson', fontsize=14)
axes4.text(0.32, 0.75, 'and', color='black', fontsize=14)
axes4.text(0.395, 0.75, 'negative in 2015 - 2016.', color='crimson', fontsize=14)
axes4.text(0, 0.7, 'Average growth rates by period:', color='crimson', fontsize=14)
axes4.text(0, 0.65, f'* 1980-1984 - {round(statistics.mean(list_EU[:5]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.6, f'* 1985-1989 - {round(statistics.mean(list_EU[5:10]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.55, f'* 1990-1994 - {round(statistics.mean(list_EU[10:15]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.5, f'* 1995-1999 - {round(statistics.mean(list_EU[15:20]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.45, f'* 2000-2004 - {round(statistics.mean(list_EU[20:25]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.4, f'* 2005-2009 -{round(statistics.mean(list_EU[25:30]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.35, f'* 2010-2016 - {round(statistics.mean(list_EU[30:]),2)}%.', color='black', fontsize=14)
fig.show()

In [ ]:
fig = plt.figure(facecolor='whitesmoke')
sns.set_style('white')
axes1 = fig.add_axes([0, 0, 1.5, 1.5]) 
axes2 = fig.add_axes([1.6, 0, 1, 1.5]) 
axes3 = fig.add_axes([0, -1.8, 1.5, 1.5]) 
axes4 = fig.add_axes([1.6, -1.8, 1, 1.5]) 

sns.lineplot(x=sales_by_year['Year'], y=sales_by_year['JP_Sales'],color='crimson',lw=4, ax=axes1)
axes1.scatter(x=[1996, 2003, 2006], y=[57.44,34.20, 73.73], color='black', lw=4)
axes1.set_facecolor('whitesmoke')
axes1.set_xlabel('Year', color='black', fontsize=18)
axes1.set_ylabel('Sales', color='black', fontsize=18)
axes1.text(1979, 70, 'Sales in', color='black', fontsize=20, fontweight='bold')
axes1.text(1985, 70, 'Japan', color='crimson', fontsize=24, fontweight='bold')

# 1st max
axes1.annotate('', xy=(1995, 57.44), xytext=(1988, 35.6),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=-.3'))
axes1.annotate('1st max sales in', xy=(1982, 32.6), xytext=(1982, 32.6), color='black', fontweight='bold', fontsize=12)
axes1.annotate('1996', xy=(1989, 32.6), xytext=(1989, 32.6), color='crimson', fontweight='bold', fontsize=12)

# 2nd max
axes1.annotate('', xy=(2007, 74.23), xytext=(2016, 64.73),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.3'))
axes1.annotate('2nd max sales in', xy=(2009.5, 61.6), xytext=(2009.5, 61.6), color='black', fontweight='bold', fontsize=12)
axes1.annotate('2006', xy=(2014.4, 59), xytext=(2014.4, 59), color='crimson', fontweight='bold', fontsize=12)

# downturns
axes1.annotate('', xy=(2003, 33.6), xytext=(2003, 20.2),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=0'))
axes1.annotate('Downturn in', xy=(1997, 18.8), xytext=(1999, 18.8), color='black', fontweight='bold', fontsize=12)
axes1.annotate('2003', xy=(2004.3, 18.8), xytext=(2004.3, 18.8), color='crimson', fontweight='bold', fontsize=12)

# conclusion № 1
axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.8, 'On the chart we see', color='black', fontsize=14)
axes2.text(0.34, 0.8, '2 periods of growth', color='crimson', fontsize=14)
axes2.text(0.67, 0.8, 'the maximum', color='black', fontsize=14)
axes2.text(0, 0.75, 'points of which were', color='black', fontsize=14)
axes2.text(0.35, 0.75, 'in 1996 and 2006.', color='crimson', fontsize=14)
axes2.text(0, 0.7, 'After 1993 was', color='black', fontsize=14)
axes2.text(0.255, 0.7, 'strong donwturn,', color='crimson', fontsize=14)
axes2.text(0.545, 0.7, 'which stopped', color='black', fontsize=14)
axes2.text(0, 0.65, 'in 2003.', color='crimson', fontsize=14)
axes2.text(0, 0.6, '----------------------------------------------------------------------------------------', color='black', fontsize=14)
axes2.text(0, 0.55, 'In comparison with the previous charts', color='black', fontsize=14)
axes2.text(0.65, 0.55, 'this chart is', color='crimson', fontsize=14)
axes2.text(0, 0.5, 'more variable.', color='crimson', fontsize=14)
axes2.text(0.25, 0.5, 'We observe', color='black', fontsize=14)
axes2.text(0.45, 0.5, '2 "ridges" with maximum sales.', color='crimson', fontsize=14)

# calculation of sales growth rates
list_JP = []
for n in range(4, 37):
    d = ((sales_by_year['JP_Sales'][n] - sales_by_year['JP_Sales'][n-1])/sales_by_year['JP_Sales'][n-1])*100
    list_JP.append(d)

# visualisation growth rates sales in EU by years
colors = ['crimson' if _ > 0 else 'darksalmon' for _ in list_JP]
axes3.bar(height=list_JP, x=sales_by_year['Year'][range(4, 37)], color=colors)
for p in axes3.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    if height >= 0:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.02), ha='center')
    elif height <= -40:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.1), ha='center')
    else:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.4), ha='center')
axes3.set_facecolor('whitesmoke')
axes3.set_xlabel('Year', fontsize=18, color='black')
axes3.set_ylabel('Growth rate, %', fontsize=18, color='black')
axes3.text(1997, 95, 'Growth rate sales(%) in', color='black', fontsize=20, fontweight='bold')
axes3.text(2007.68, 83, 'Japan', color='crimson', fontsize=24, fontweight='bold')

# max +
axes3.annotate('', xy=(1990, 97), xytext=(1986, 73),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=-.3'))
axes3.annotate('Max', xy=(1985, 69), xytext=(1985, 69), color='black', fontweight='bold', fontsize=12)
axes3.annotate('+', xy=(1986.5, 69), xytext=(1986.5, 69), color='crimson', fontweight='bold', fontsize=24)

# max -
axes3.annotate('', xy=(2015, -60), xytext=(2010, -52),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.3'))
axes3.annotate('Max', xy=(2009, -51), xytext=(2009, -51), color='black', fontweight='bold', fontsize=12)
axes3.annotate('-', xy=(2010.8, -51), xytext=(2010.8, -51), color='darksalmon', fontweight='bold', fontsize=24)

# conclusion № 2
axes4.set_facecolor('whitesmoke')
axes4.axis('off')
axes4.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes4.text(0, 0.8, 'From chart we can see that', color='black', fontsize=14)
axes4.text(0.455, 0.8, 'maximum positive growth', color='crimson', fontsize=14)
axes4.text(0, 0.75, 'was', color='black', fontsize=14)
axes4.text(0.07, 0.75, 'in 1991 - 1992,', color='crimson', fontsize=14)
axes4.text(0.32, 0.75, 'and', color='black', fontsize=14)
axes4.text(0.395, 0.75, 'negative in 2015 - 2016.', color='crimson', fontsize=14)
axes4.text(0, 0.7, 'Average growth rates by period:', color='crimson', fontsize=14)
axes4.text(0, 0.65, f'* 1983-1984 - {round(statistics.mean(list_JP[:1]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.6, f'* 1985-1989 - {round(statistics.mean(list_JP[1:6]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.55, f'* 1990-1994 - {round(statistics.mean(list_JP[6:11]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.5, f'* 1995-1999 - {round(statistics.mean(list_JP[11:16]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.45, f'* 2000-2004 - {round(statistics.mean(list_JP[16:21]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.4, f'* 2005-2009 - {round(statistics.mean(list_JP[21:26]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.35, f'* 2010-2016 - {round(statistics.mean(list_JP[26:]),2)}%.', color='black', fontsize=14)
fig.show()


In [ ]:
fig = plt.figure(facecolor='whitesmoke')
sns.set_style('white')
axes1 = fig.add_axes([0, 0, 1.5, 1.5]) 
axes2 = fig.add_axes([1.6, 0, 1, 1.5]) 
axes3 = fig.add_axes([0, -1.8, 1.5, 1.5]) 
axes4 = fig.add_axes([1.6, -1.8, 1, 1.5]) 

sns.lineplot(x=sales_by_year['Year'], y=sales_by_year['Other_Sales'],color='crimson',lw=4, ax=axes1)
axes1.scatter(x=[1999, 2003, 2005, 2008, 2013, 2014], y=[10.05, 26.01, 40.55, 82.39, 39.82, 40.02], color='black', lw=4)
axes1.set_facecolor('whitesmoke')
axes1.set_xlabel('Year', color='black', fontsize=18)
axes1.set_ylabel('Sales', color='black', fontsize=18)
axes1.text(1979, 80, 'Sales in', color='black', fontsize=20, fontweight='bold')
axes1.text(1985, 80, 'Other countries', color='crimson', fontsize=24, fontweight='bold')

# growth
axes1.annotate('', xy=(2006, 81), xytext=(1995, 7),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('Strong growth with 3 downturns in', xy=(1994.1, 10), xytext=(1994.1, 10), rotation=62.5, color='black', fontweight='bold', fontsize=12)
axes1.annotate('1999, 2003', xy=(2001, 55.5), xytext=(2001, 55.5), rotation=62.5, color='crimson', fontweight='bold', fontsize=12)
axes1.annotate('and', xy=(2003.2, 70.6), xytext=(2003.2, 70.6), rotation=62.5, color='black', fontweight='bold', fontsize=12)
axes1.annotate('2005', xy=(2004, 76.4), xytext=(2004, 76.4), rotation=62.5, color='crimson', fontweight='bold', fontsize=12)

# stagnation
axes1.annotate('', xy=(2012.9, 38.5), xytext=(2008, 21),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('', xy=(2013.9, 38.5), xytext=(2012, 21),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('Stagnation in', xy=(2001.5, 18.5), xytext=(2001.5, 18.5), color='black', fontweight='bold', fontsize=12)
axes1.annotate('2013', xy=(2007.35, 18.5), xytext=(2007.35, 18.5),color='crimson', fontweight='bold', fontsize=12)
axes1.annotate('and', xy=(2009.5, 18.5), xytext=(2009.5, 18.5),color='black', fontweight='bold', fontsize=12)
axes1.annotate('2014', xy=(2011.2, 18.5), xytext=(2011.2, 18.5),color='crimson', fontweight='bold', fontsize=12)

# max
axes1.annotate('', xy=(2009, 84), xytext=(2015, 67),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.3'))
axes1.annotate('Maximum sales in', xy=(2009.8, 65), xytext=(2009.8, 65), color='black', fontweight='bold', fontsize=12)
axes1.annotate('2009', xy=(2015.2, 62), xytext=(2015.2, 62), color='crimson', fontweight='bold', fontsize=12)

# conclusion № 1
axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.8, 'Strong growth', color='crimson', fontsize=14)
axes2.text(0.24, 0.8, 'began', color='black', fontsize=14)
axes2.text(0.353, 0.8, 'after 1995', color='crimson', fontsize=14)
axes2.text(0.535, 0.8, 'and continued', color='black', fontsize=14)
axes2.text(0.787, 0.8, 'until 2008.', color='crimson', fontsize=14)
axes2.text(0, 0.75, 'Else we can see', color='black', fontsize=14)
axes2.text(0.27, 0.75, '3 downturns in 1999, 2003 and 2005.', color='crimson', fontsize=14)
axes2.text(0, 0.7, 'We observe that', color='black', fontsize=14)
axes2.text(0.28, 0.7, 'sales decreased after 2008.', color='crimson', fontsize=14)
axes2.text(0, 0.65, 'Sales downturns stopped in 2013-2014', color='crimson', fontsize=14)
axes2.text(0.64, 0.65, 'and then began again', color='black', fontsize=14)
axes2.text(0, 0.6, '----------------------------------------------------------------------------------------', color='black', fontsize=14)
axes2.text(0, 0.55, 'On this chart we see familiar picture, which was', color='black', fontsize=14)
axes2.text(0, 0.50, 'in North America and Europe.', color='black', fontsize=14)


# calculation of sales growth rates
list_Other = []
for n in range(1, 37):
    d = ((sales_by_year['Other_Sales'][n] - sales_by_year['Other_Sales'][n-1])/sales_by_year['Other_Sales'][n-1])*100
    list_Other.append(d)

# visualisation growth rates sales in Other by years
colors = ['crimson' if _ > 0 else 'darksalmon' for _ in list_Other]
axes3.bar(height=list_Other, x=sales_by_year['Year'][range(1, 37)], color=colors)
for p in axes3.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    if height >= 0:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.02), ha='center')
    elif height <= -55:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.2), ha='center')
    else:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.4), ha='center')
axes3.set_facecolor('whitesmoke')
axes3.set_xlabel('Year', fontsize=18, color='black')
axes3.set_ylabel('Growth rate, %', fontsize=18, color='black')
axes3.text(1997, 365, 'Growth rate sales(%) in', color='black', fontsize=20, fontweight='bold')
axes3.text(2000.5, 330, 'Other countries', color='crimson', fontsize=24, fontweight='bold')

# max +
axes3.annotate('', xy=(1983, 401), xytext=(1980, 322),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=-.3'))
axes3.annotate('Max', xy=(1979, 312), xytext=(1979, 312), color='black', fontweight='bold', fontsize=12)
axes3.annotate('+', xy=(1981, 312), xytext=(1981, 312), color='crimson', fontweight='bold', fontsize=24)

# max -
axes3.annotate('', xy=(1987, -91), xytext=(1995, -72),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=-.3'))
axes3.annotate('Max', xy=(1994, -62), xytext=(1994, -62), color='black', fontweight='bold', fontsize=12)
axes3.annotate('-', xy=(1996, -62), xytext=(1996, -62), color='darksalmon', fontweight='bold', fontsize=24)

# conclusion № 2
axes4.set_facecolor('whitesmoke')
axes4.axis('off')
axes4.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes4.text(0, 0.8, 'Maximum positive growth', color='crimson', fontsize=14)
axes4.text(0.445, 0.8, '-', color='black', fontsize=14)
axes4.text(0.475, 0.8, '1983-1984.', color='crimson', fontsize=14)
axes4.text(0.0, 0.75, 'Maximum negative growth', color='crimson', fontsize=14)
axes4.text(0.445, 0.75, '-', color='black', fontsize=14)
axes4.text(0.475, 0.75, '1983-1984.', color='crimson', fontsize=14)
axes4.text(0, 0.7, 'Average growth rates by period:', color='crimson', fontsize=14)
axes4.text(0, 0.65, f'* 1980-1984 - {round(statistics.mean(list_Other[:5]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.6, f'* 1985-1989 - {round(statistics.mean(list_Other[5:10]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.55, f'* 1990-1994 - {round(statistics.mean(list_Other[10:15]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.5, f'* 1995-1999 - {round(statistics.mean(list_Other[15:20]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.45, f'* 2000-2004 - {round(statistics.mean(list_Other[20:25]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.4, f'* 2005-2009 - {round(statistics.mean(list_Other[25:30]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.35, f'* 2010-2016 - {round(statistics.mean(list_Other[30:]),2)}%.', color='black', fontsize=14)
fig.show()

In [ ]:
fig = plt.figure(facecolor='whitesmoke')
sns.set_style('white')
axes1 = fig.add_axes([0, 0, 1.5, 1.5]) 
axes2 = fig.add_axes([1.6, 0, 1, 1.5]) 
axes3 = fig.add_axes([0, -1.8, 1.5, 1.5]) 
axes4 = fig.add_axes([1.6, -1.8, 1, 1.5]) 

sns.lineplot(x=sales_by_year['Year'], y=sales_by_year['Global_Sales'],color='crimson',lw=4, ax=axes1)
axes1.scatter(x=[2000, 2003, 2008, 2012, 2013], y=[ 201.56, 357.85, 678.90, 363.53, 368.11], color='black', lw=4)
axes1.set_facecolor('whitesmoke')
axes1.set_xlabel('Year', fontsize=18, color='black')
axes1.set_ylabel('Sales', fontsize=18, color='black')
axes1.text(1979, 640, 'Global sales', color='crimson', fontsize=24, fontweight='bold')

# growth
axes1.annotate('', xy=(2007, 681), xytext=(1992, 100),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('Strong growth with 2 downturns in', xy=(1991.2, 135), xytext=(1991.2, 135), rotation=54, color='black', fontweight='bold', fontsize=12)
axes1.annotate('2000', xy=(2000, 476), xytext=(2000, 476), rotation=54, color='crimson', fontweight='bold', fontsize=12)
axes1.annotate('and', xy=(2001.3, 529), xytext=(2001.3, 529), rotation=54, color='black', fontweight='bold', fontsize=12)
axes1.annotate('2003', xy=(2002.3, 570), xytext=(2002.3, 570), rotation=54, color='crimson', fontweight='bold', fontsize=12)

# stagnation
axes1.annotate('', xy=(2011.9, 357), xytext=(2010, 183),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('', xy=(2012.9, 357), xytext=(2012, 183),
              arrowprops=dict(color='black', arrowstyle='->'))
axes1.annotate('Stagnation in', xy=(2001, 165), xytext=(2001, 165), color='black', fontweight='bold', fontsize=12)
axes1.annotate('2013', xy=(2007, 165), xytext=(2007, 165),color='crimson', fontweight='bold', fontsize=12)
axes1.annotate('and', xy=(2009.3, 165), xytext=(2009.3, 165),color='black', fontweight='bold', fontsize=12)
axes1.annotate('2014', xy=(2011.2, 165), xytext=(2011.2, 165),color='crimson', fontweight='bold', fontsize=12)

# # max
axes1.annotate('', xy=(2008, 690), xytext=(2015, 580),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.6'))
axes1.annotate('Maximum sales', xy=(2011, 565), xytext=(2011, 565), color='black', fontweight='bold', fontsize=12)
axes1.annotate('in 2008', xy=(2014.2, 540), xytext=(2014.2, 540), color='crimson', fontweight='bold', fontsize=12)

# conclusion № 1
axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.8, 'Between 1993 and 2008', color='crimson', fontsize=14)
axes2.text(0.422, 0.8, 'was', color='black', fontsize=14)
axes2.text(0.505, 0.8, 'strong growth', color='crimson', fontsize=14)
axes2.text(0, 0.75, 'with', color='black', fontsize=14)
axes2.text(0.08, 0.75, '2 downturns in 2000 and 2003.', color='crimson', fontsize=14)
axes2.text(0, 0.7, 'After 2008', color='crimson', fontsize=14)
axes2.text(0.185, 0.7, 'we see', color='black', fontsize=14)
axes2.text(0.32, 0.7, 'decreased sales.', color='crimson', fontsize=14)
axes2.text(0, 0.65, 'Also we observe', color='black', fontsize=14)
axes2.text(0.278, 0.65, 'stagnation in 2013-2014.', color='crimson', fontsize=14)

# calculation of sales growth rates
list_Global = []
for n in range(1, 37):
    d = ((sales_by_year['Global_Sales'][n] - sales_by_year['Global_Sales'][n-1])/sales_by_year['Global_Sales'][n-1])*100
    list_Global.append(d)

# visualisation growth rates sales in Other by years
colors = ['crimson' if _ > 0 else 'darksalmon' for _ in list_Global]
axes3.bar(height=list_Global, x=sales_by_year['Year'][range(1, 37)], color=colors)
for p in axes3.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    if height >= 0:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.02), ha='center')
    elif height <= -55:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.1), ha='center')
    else:
        axes3.annotate('{:.0f}'.format(height), (x + width/2, y + height*1.4), ha='center')
axes3.set_facecolor('whitesmoke')
axes3.set_xlabel('Year', fontsize=18, color='black')
axes3.set_ylabel('Growth rate, %', fontsize=18, color='black')
axes3.text(1994, 180, 'Global growth rate sales(%)', color='crimson', fontsize=24, fontweight='bold')

# max +
axes3.annotate('', xy=(1981.5, 215), xytext=(1989, 180),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.3'))
axes3.annotate('Max', xy=(1987, 172), xytext=(1987, 172), color='black', fontweight='bold', fontsize=12)
axes3.annotate('+', xy=(1988.8, 171), xytext=(1988.8, 171), color='crimson', fontweight='bold', fontsize=24)

# max -
axes3.annotate('', xy=(2015, -73), xytext=(2008, -57),
               arrowprops=dict(color='black', arrowstyle='->', connectionstyle='arc3,rad=.3'))
axes3.annotate('Max', xy=(2006, -46), xytext=(2006, -46), color='black', fontweight='bold', fontsize=12)
axes3.annotate('-', xy=(2007.8, -46), xytext=(2007.8, -46), color='darksalmon', fontweight='bold', fontsize=24)

# conclusion № 2
axes4.set_facecolor('whitesmoke')
axes4.axis('off')
axes4.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes4.text(0, 0.8, 'Maximum positive growth', color='crimson', fontsize=14)
axes4.text(0.438, 0.8, 'was', color='black', fontsize=14)
axes4.text(0.52, 0.8, 'in 1980-1981,', color='crimson', fontsize=14)
axes4.text(0.76, 0.8, 'and', color='black', fontsize=14)
axes4.text(0, 0.75, 'maximum negative growth', color='crimson', fontsize=14)
axes4.text(0.45, 0.75, '-', color='black', fontsize=14)
axes4.text(0.48, 0.75, 'in 2015-2016.', color='crimson', fontsize=14)
axes4.text(0, 0.7, 'Average growth rates by period:', color='crimson', fontsize=14)
axes4.text(0, 0.65, f'* 1980-1984 - {round(statistics.mean(list_Global[:5]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.6, f'* 1985-1989 - {round(statistics.mean(list_Global[5:10]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.55, f'* 1990-1994 - {round(statistics.mean(list_Global[10:15]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.5, f'* 1995-1999 - {round(statistics.mean(list_Global[15:20]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.45, f'* 2000-2004 - {round(statistics.mean(list_Global[20:25]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.4, f'* 2005-2009 - {round(statistics.mean(list_Global[25:30]),2)}%;', color='black', fontsize=14)
axes4.text(0, 0.35, f'* 2010-2016 - {round(statistics.mean(list_Global[30:]),2)}%.', color='black', fontsize=14)

fig.show()

Sprzedaż w regionach z podziałem na typy gier

In [ ]:
# NA
group_genre_na = data.groupby('Genre')['NA_Sales'].sum().reset_index()
group_genre_na['NA_Sales'] = round(group_genre_na['NA_Sales'],2)
group_genre_na['Percentages'] = group_genre_na['NA_Sales']/sum(group_genre_na['NA_Sales'])*100
group_genre_na['Percentages'] = round(group_genre_na['Percentages'],2)
group_genre_na.sort_values(by='Percentages', ascending=False, inplace=True)
group_genre_na.reset_index(inplace=True)
group_genre_na.drop('index', axis=1, inplace=True)

# EU
group_genre_eu = data.groupby('Genre')['EU_Sales'].sum().reset_index()
group_genre_eu['EU_Sales'] = round(group_genre_eu['EU_Sales'],2)
group_genre_eu['Percentages'] = group_genre_eu['EU_Sales']/sum(group_genre_eu['EU_Sales'])*100
group_genre_eu['Percentages'] = round(group_genre_eu['Percentages'],2)
group_genre_eu.sort_values(by='Percentages', ascending=False, inplace=True)
group_genre_eu.reset_index(inplace=True)
group_genre_eu.drop('index', axis=1, inplace=True)

# JP
group_genre_jp = data.groupby('Genre')['JP_Sales'].sum().reset_index()
group_genre_jp['JP_Sales'] = round(group_genre_jp['JP_Sales'],2)
group_genre_jp['Percentages'] = group_genre_jp['JP_Sales']/sum(group_genre_jp['JP_Sales'])*100
group_genre_jp['Percentages'] = round(group_genre_jp['Percentages'],2)
group_genre_jp.sort_values(by='Percentages', ascending=False, inplace=True)
group_genre_jp.reset_index(inplace=True)
group_genre_jp.drop('index', axis=1, inplace=True)

# Other
group_genre_other = data.groupby('Genre')['Other_Sales'].sum().reset_index()
group_genre_other['Other_Sales'] = round(group_genre_other['Other_Sales'],2)
group_genre_other['Percentages'] = group_genre_other['Other_Sales']/sum(group_genre_other['Other_Sales'])*100
group_genre_other['Percentages'] = round(group_genre_other['Percentages'],2)
group_genre_other.sort_values(by='Percentages', ascending=False, inplace=True)
group_genre_other.reset_index(inplace=True)
group_genre_other.drop('index', axis=1, inplace=True)

# Global
group_genre_global = data.groupby('Genre')['Global_Sales'].sum().reset_index()
group_genre_global['Global_Sales'] = round(group_genre_global['Global_Sales'],2)
group_genre_global['Percentages'] = group_genre_global['Global_Sales']/sum(group_genre_global['Global_Sales'])*100
group_genre_global['Percentages'] = round(group_genre_global['Percentages'],2)
group_genre_global.sort_values(by='Percentages', ascending=False, inplace=True)
group_genre_global.reset_index(inplace=True)
group_genre_global.drop('index', axis=1, inplace=True)


In [ ]:
list_na=[]
for n in range(len(group_genre_na['Genre'])):
    x = group_genre_na.loc[n,:]
    list_na.append(x)
color_list=[['whitesmoke', 'white', 'white']]
fig = plt.figure(facecolor='whitesmoke')
axes1 = fig.add_axes([0, 0, 1, 1]) 
axes2 = fig.add_axes([1.6, 0, 1, 1]) 

axes1.set_axis_off() 
table=axes1.table(cellColours=color_list*12,cellText = list_na, cellLoc ='left', loc ='upper left', colWidths=[0.3,0.3,0.3],
                  colLabels=group_genre_na.columns,colColours=['crimson']*3)           
table.auto_set_font_size(False) 
table.set_fontsize(16)  
table.scale(1.5, 2.7) 
axes1.text(0.1, 1.15, 'Sales by genre in', color='black', fontsize=20, fontweight='bold')
axes1.text(0.63, 1.15, 'North America', color='crimson', fontsize=24, fontweight='bold')

axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.8, 'Top 3 genres in North America:', color='crimson', fontsize=14)
axes2.text(0, 0.72, f"* {group_genre_na['Genre'][0]};", color='black', fontsize=14)
axes2.text(0, 0.64, f"* {group_genre_na['Genre'][1]};", color='black', fontsize=14)
axes2.text(0, 0.56, f"* {group_genre_na['Genre'][2]};", color='black', fontsize=14)
axes2.text(0, 0.48, f"Top 3 genres = {sum(group_genre_na['Percentages'][:3])}% of the total sales amount", color='crimson', fontsize=14)
axes2.text(0, 0.4, '3 least popular genres in North America', color='crimson', fontsize=14)
axes2.text(0, 0.32, f"* {group_genre_na['Genre'][9]};", color='black', fontsize=14)
axes2.text(0, 0.24, f"* {group_genre_na['Genre'][10]};", color='black', fontsize=14)
axes2.text(0, 0.16, f"* {group_genre_na['Genre'][11]};", color='black', fontsize=14)
axes2.text(0, 0.08, f"3 least popular genres = {sum(group_genre_na['Percentages'][9:])}% of the total sales amount", color='crimson', fontsize=14)

fig.show()

In [ ]:
list_eu=[]
for n in range(len(group_genre_eu['Genre'])):
    x = group_genre_eu.loc[n,:]
    list_eu.append(x)
color_list=[['whitesmoke', 'white', 'white']]
fig = plt.figure(facecolor='whitesmoke')
axes1 = fig.add_axes([0, 0, 1, 1]) 
axes2 = fig.add_axes([1.6, 0, 1, 1]) 

axes1.set_axis_off() 
table=axes1.table(cellColours=color_list*12,cellText = list_eu, cellLoc ='left', loc ='upper left', colWidths=[0.3,0.3,0.3],
                  colLabels=group_genre_eu.columns,colColours=['crimson']*3)      
table.auto_set_font_size(False) 
table.set_fontsize(16)  
table.scale(1.5, 2.7) 
axes1.text(0.1, 1.15, 'Sales by genre in', color='black', fontsize=20, fontweight='bold')
axes1.text(0.63, 1.15, 'Europe', color='crimson', fontsize=24, fontweight='bold')

axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.8, 'Top 3 genres in Europe:', color='crimson', fontsize=14)
axes2.text(0, 0.72, f"* {group_genre_eu['Genre'][0]};", color='black', fontsize=14)
axes2.text(0, 0.64, f"* {group_genre_eu['Genre'][1]};", color='black', fontsize=14)
axes2.text(0, 0.56, f"* {group_genre_eu['Genre'][2]};", color='black', fontsize=14)
axes2.text(0, 0.48, f"Top 3 genres = {round(sum(group_genre_eu['Percentages'][:3]),2)}% of the total sales amount", color='crimson', fontsize=14)
axes2.text(0, 0.4, '3 least popular genres in Europe', color='crimson', fontsize=14)
axes2.text(0, 0.32, f"* {group_genre_eu['Genre'][9]};", color='black', fontsize=14)
axes2.text(0, 0.24, f"* {group_genre_eu['Genre'][10]};", color='black', fontsize=14)
axes2.text(0, 0.16, f"* {group_genre_eu['Genre'][11]};", color='black', fontsize=14)
axes2.text(0, 0.08, f"3 least popular genres = {sum(group_genre_eu['Percentages'][9:])}% of the total sales amount", color='crimson', fontsize=14)

fig.show()

In [ ]:
list_jp=[]
for n in range(len(group_genre_jp['Genre'])):
    x = group_genre_jp.loc[n,:]
    list_jp.append(x)
color_list=[['whitesmoke', 'white', 'white']]
fig = plt.figure(facecolor='whitesmoke')
axes1 = fig.add_axes([0, 0, 1, 1]) 
axes2 = fig.add_axes([1.6, 0, 1, 1]) 

axes1.set_axis_off() 
table=axes1.table(cellColours=color_list*12,cellText = list_jp, cellLoc ='left', loc ='upper left', colWidths=[0.3,0.3,0.3],
                  colLabels=group_genre_jp.columns,colColours=['crimson']*3)      
table.auto_set_font_size(False) 
table.set_fontsize(16)  
table.scale(1.5, 2.7) 
axes1.text(0.1, 1.15, 'Sales by genre in', color='black', fontsize=20, fontweight='bold')
axes1.text(0.63, 1.15, 'Japan', color='crimson', fontsize=24, fontweight='bold')

axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.8, 'Top 3 genres in Japan:', color='crimson', fontsize=14)
axes2.text(0, 0.72, f"* {group_genre_jp['Genre'][0]};", color='black', fontsize=14)
axes2.text(0, 0.64, f"* {group_genre_jp['Genre'][1]};", color='black', fontsize=14)
axes2.text(0, 0.56, f"* {group_genre_jp['Genre'][2]};", color='black', fontsize=14)
axes2.text(0, 0.48, f"Top 3 genres = {round(sum(group_genre_jp['Percentages'][:3]),2)}% of the total sales amount", color='crimson', fontsize=14)
axes2.text(0, 0.4, '3 least popular genres in Japan', color='crimson', fontsize=14)
axes2.text(0, 0.32, f"* {group_genre_jp['Genre'][9]};", color='black', fontsize=14)
axes2.text(0, 0.24, f"* {group_genre_jp['Genre'][10]};", color='black', fontsize=14)
axes2.text(0, 0.16, f"* {group_genre_jp['Genre'][11]};", color='black', fontsize=14)
axes2.text(0, 0.08, f"3 least popular genres = {sum(group_genre_jp['Percentages'][9:])}% of the total sales amount", color='crimson', fontsize=14)

fig.show()

In [ ]:
list_other=[]
for n in range(len(group_genre_other['Genre'])):
    x = group_genre_other.loc[n,:]
    list_other.append(x)
color_list=[['whitesmoke', 'white', 'white']]
fig = plt.figure(facecolor='whitesmoke')
axes1 = fig.add_axes([0, 0, 1, 1]) 
axes2 = fig.add_axes([1.6, 0, 1, 1]) 

axes1.set_axis_off() 
table=axes1.table(cellColours=color_list*12,cellText = list_other, cellLoc ='left', loc ='upper left', colWidths=[0.3,0.3,0.3],
                  colLabels=group_genre_other.columns,colColours=['crimson']*3)      
table.auto_set_font_size(False) 
table.set_fontsize(16)  
table.scale(1.5, 2.7) 
axes1.text(0.1, 1.15, 'Sales by genre in', color='black', fontsize=20, fontweight='bold')
axes1.text(0.63, 1.15, 'Other countries', color='crimson', fontsize=24, fontweight='bold')

axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.8, 'Top 3 genres in Other countries:', color='crimson', fontsize=14)
axes2.text(0, 0.72, f"* {group_genre_other['Genre'][0]};", color='black', fontsize=14)
axes2.text(0, 0.64, f"* {group_genre_other['Genre'][1]};", color='black', fontsize=14)
axes2.text(0, 0.56, f"* {group_genre_other['Genre'][2]};", color='black', fontsize=14)
axes2.text(0, 0.48, f"Top 3 genres = {round(sum(group_genre_other['Percentages'][:3]),2)}% of the total sales amount", color='crimson', fontsize=14)
axes2.text(0, 0.4, '3 least popular genres in Other countries', color='crimson', fontsize=14)
axes2.text(0, 0.32, f"* {group_genre_other['Genre'][9]};", color='black', fontsize=14)
axes2.text(0, 0.24, f"* {group_genre_other['Genre'][10]};", color='black', fontsize=14)
axes2.text(0, 0.16, f"* {group_genre_other['Genre'][11]};", color='black', fontsize=14)
axes2.text(0, 0.08, f"3 least popular genres = {sum(group_genre_other['Percentages'][9:])}% of the total sales amount", color='crimson', fontsize=14)

fig.show()

In [ ]:
list_global=[]
for n in range(len(group_genre_global['Genre'])):
    x = group_genre_global.loc[n,:]
    list_global.append(x)
color_list=[['whitesmoke', 'white', 'white']]
fig = plt.figure(facecolor='whitesmoke')
axes1 = fig.add_axes([0, 0, 1, 1]) 
axes2 = fig.add_axes([1.6, 0, 1, 1]) 

axes1.set_axis_off() 
table=axes1.table(cellColours=color_list*12,cellText = list_global, cellLoc ='left', loc ='upper left', colWidths=[0.3,0.3,0.3],
                  colLabels=group_genre_global.columns,colColours=['crimson']*3)      
table.auto_set_font_size(False) 
table.set_fontsize(16)  
table.scale(1.5, 2.7) 
axes1.text(0.1, 1.15, 'Global sales by genre', color='crimson', fontsize=24, fontweight='bold')

axes2.set_facecolor('whitesmoke')
axes2.axis('off')
axes2.text(0.2, 0.9, 'Conclusion', color='crimson', fontsize=24, fontweight='bold')
axes2.text(0, 0.8, 'Top 3 genres:', color='crimson', fontsize=14)
axes2.text(0, 0.72, f"* {group_genre_global['Genre'][0]};", color='black', fontsize=14)
axes2.text(0, 0.64, f"* {group_genre_global['Genre'][1]};", color='black', fontsize=14)
axes2.text(0, 0.56, f"* {group_genre_global['Genre'][2]};", color='black', fontsize=14)
axes2.text(0, 0.48, f"Top 3 genres = {round(sum(group_genre_global['Percentages'][:3]),2)}% of the total sales amount", color='crimson', fontsize=14)
axes2.text(0, 0.4, '3 least popular genres', color='crimson', fontsize=14)
axes2.text(0, 0.32, f"* {group_genre_global['Genre'][9]};", color='black', fontsize=14)
axes2.text(0, 0.24, f"* {group_genre_global['Genre'][10]};", color='black', fontsize=14)
axes2.text(0, 0.16, f"* {group_genre_global['Genre'][11]};", color='black', fontsize=14)
axes2.text(0, 0.08, f"3 least popular genres = {sum(group_genre_global['Percentages'][9:])}% of the total sales amount", color='crimson', fontsize=14)

fig.show()

Najdrożej sprzedana gra w północnej Ameryce:

In [ ]:
north_usa_highest_sold_game = data.NA_Sales.max()
data[data["NA_Sales"] == north_usa_highest_sold_game][["Name","NA_Sales"]]

# Na podstawie tak obszernej i dokładnej analizy zmiennych ilościowych oraz jakościowych, określono trzy hipotezy którym można się dokładniej przyjrzeć w tej pracy. 

* **Hipoteza 1:** Znaczną część sprzedaży globalnej stanowi sprzedaż gier na rynek Amerykański
* **Hipoteza 2:** Sprzedaż gier na PS2 rośnie z każdym rokiem na rynku japońskim
* **Hipoteza 3:** Dominującym gatunkiem gier w Europie jest gatunek gry akcji

In [ ]:
sns.histplot(x=data["Platform"],hue=data["Year"])
data[["Platform", "Year", "Rank"]].groupby(["Platform", "Year"]).count()

Tabele wielodzielcze dla zmiennych jakościowych dla hipotezy 1. Przez to, że kolumna NA_Sales ma wiele różnych wartości, postanowiłem podzielić ją na przedziały

In [ ]:
hipo_1 = data.copy()
hipo_1.NA_Sales = hipo_1.NA_Sales.astype(str)
hipo_1.NA_Sales.values[data["NA_Sales"].values < 5] = "<5"
hipo_1.NA_Sales.values[(data["NA_Sales"].values < 15) & (data["NA_Sales"].values >= 5)] = "5<=x<15"
hipo_1.NA_Sales.values[(data["NA_Sales"].values < 25) & (data["NA_Sales"].values >= 15)] = "15<=x<25"
hipo_1.NA_Sales.values[(data["NA_Sales"].values <= 45) & (data["NA_Sales"].values >= 25)] = "25<=x<=45"

for col in quali_cols:
    print(pd.crosstab(hipo_1[col], hipo_1['NA_Sales'], normalize="index"))
    print("\n\n")

W powyższych tabelach wielodzielczych widzimy udział w statystykach gier o sprzedaży 25-45 mln egzemplarzy. 

Wizualizacja wyników: 

In [ ]:
fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quali_cols):
    axes[i].set_title(f"Wykres histogramu skategoryzowanego {col}")
    sns.histplot(data = hipo_1, x=hipo_1[col], hue='NA_Sales', ax=axes[i], multiple="dodge")

Dla Hipotezy 2 stworzono podobne tabele wielodzielcze

In [ ]:
hipo_2 = data.copy()
hipo_2.JP_Sales = hipo_1.NA_Sales.astype(str)
hipo_2.JP_Sales.values[data["JP_Sales"].values < 5] = "<5"
hipo_2.JP_Sales.values[(data["JP_Sales"].values < 15) & (data["JP_Sales"].values >= 5)] = "5<=x<15"
hipo_2.JP_Sales.values[(data["JP_Sales"].values < 25) & (data["JP_Sales"].values >= 15)] = "15<=x<25"
hipo_2.JP_Sales.values[(data["JP_Sales"].values <= 45) & (data["JP_Sales"].values >= 25)] = "25<=x<=45"

for col in quali_cols:
    print(pd.crosstab(hipo_2[col], hipo_2['JP_Sales'], normalize="index"))
    print("\n\n")

W powyższych tabelach widzimy zależność, że z każdym rokiem rośnie liczba sprzedanych gier. Jednak ten trend przestaje obowiązywać po 2010 roku.

Wizualizacja wyników:

In [ ]:
fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quali_cols):
    axes[i].set_title(f"Wykres histogramu skategoryzowanego {col}")
    sns.histplot(data = hipo_2, x=hipo_2[col], hue='JP_Sales', ax=axes[i], multiple="dodge")

Oraz podobnie dla hipotezy 3

In [ ]:
hipo_3 = data.copy()
hipo_3.EU_Sales = hipo_1.NA_Sales.astype(str)
hipo_3.EU_Sales.values[data["EU_Sales"].values < 5] = "<5"
hipo_3.EU_Sales.values[(data["EU_Sales"].values < 15) & (data["EU_Sales"].values >= 5)] = "5<=x<15"
hipo_3.EU_Sales.values[(data["EU_Sales"].values < 25) & (data["EU_Sales"].values >= 15)] = "15<=x<25"
hipo_3.EU_Sales.values[(data["EU_Sales"].values <= 45) & (data["EU_Sales"].values >= 25)] = "25<=x<=45"

for col in quali_cols:
    print(pd.crosstab(hipo_3[col], hipo_3['EU_Sales'], normalize="index"))
    print("\n\n")

Tabele rozdzielcze nie do końca pokazują, że gry akcji stanowią największą część w Europie. Wyprzedzają je m.in gry strzelankowe ze sprzedażą 0.23% egzemplarzy pomiędzy 5 a 15 mln sztuk

Wizualizacja wyników

In [ ]:
fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quali_cols):
    axes[i].set_title(f"Wykres histogramu skategoryzowanego {col}")
    sns.histplot(data = hipo_3, x=hipo_3[col], hue='EU_Sales', ax=axes[i], multiple="dodge")

Wykonanie macierzy korelacji

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(data.corr(), annot=True, ax=ax)
data.corr()

W analizie korelacji możemy wykluczyć pole "Rank" gdyż jest to traktowane jako ID. Zatem korelacja występuje pomiędzy:
* "Year" a "Sales". Są to słabe korelacje, częściowo ujemne, częściowo pozorne. Jednak pole "Year" nie może zostać poddane korelacji gdyż wraz ze wzrostem sprzedaży, rosłaby wartość roku. To jest możliwe ale nie w każdym przypadku a na pewno nie jest to czysto logiczne
* "Sales" a "Sales". Pomiędzy wszystkimi wartościami sprzedaży istnieje korelacja dodatnia co jest jak najbardziej sensowne. W momemncie gdy rośnie sprzedaż w którymś regionie świata, automatycznie rośnie sprzedaż w aspekcie całego globu

Najmocniejszą korelacją wykazuje się para "NA_Sales" oraz "Global_Sales" (0.94) co jest bardzo silną korelacją. Oznacza to największy wpływ na wzrost globalnej sprzedaży artykułów na rynek Ameryki Północnej

Test niezależności przy użyciu chi^2 przy założeniu poziomu istotności alfa=0.05, zakładając za hipotezę zerową, że zmienne są niezależne

Test niezależności dla hipotezy 1 (NA_Sales)

In [ ]:
cols_to_drop = ["Rank", "NA_Sales"]
for col in data.columns:
    if col not in cols_to_drop:
        contigency = pd.crosstab(data[col], data["NA_Sales"])
        chi,p_value,degrees_of_freedom , expected_freq = chi2_contingency(contigency)
        print(f"Dla kolumny {col} wartość p testu niezależności wynosi {p_value}")
        if p_value <= 0.05:
            print(f"Wartość p jest mniejsza dla założonego poziomu istotności co pozwala na odrzucenie hipotezy zerowej - zmienne są zależne\n")
        else:
            print(f"Wartość p jest większa dla założonego poziomu istotności co potwierdza hipotezę zerową - zmienne są niezależne\n")

Test niezależności dla hipotezy 2 (JP_Sales)

In [ ]:
cols_to_drop = ["Rank", "JP_Sales"]
for col in data.columns:
    if col not in cols_to_drop:
        contigency = pd.crosstab(data[col], data["JP_Sales"])
        chi,p_value,degrees_of_freedom , expected_freq = chi2_contingency(contigency)
        print(f"Dla kolumny {col} wartość p testu niezależności wynosi {p_value}")
        if p_value <= 0.05:
            print(f"Wartość p jest mniejsza dla założonego poziomu istotności co pozwala na odrzucenie hipotezy zerowej - zmienne są zależne\n")
        else:
            print(f"Wartość p jest większa dla założonego poziomu istotności co potwierdza hipotezę zerową - zmienne są niezależne\n")

Test niezależności dla hipotezy 3 (EU_Sales)

In [ ]:
cols_to_drop = ["Rank", "EU_Sales"]
for col in data.columns:
    if col not in cols_to_drop:
        contigency = pd.crosstab(data[col], data["EU_Sales"])
        chi,p_value,degrees_of_freedom , expected_freq = chi2_contingency(contigency)
        print(f"Dla kolumny {col} wartość p testu niezależności wynosi {p_value}")
        if p_value <= 0.05:
            print(f"Wartość p jest mniejsza dla założonego poziomu istotności co pozwala na odrzucenie hipotezy zerowej - zmienne są zależne\n")
        else:
            print(f"Wartość p jest większa dla założonego poziomu istotności co potwierdza hipotezę zerową - zmienne są niezależne\n")

Wykresy ramka-wąsy dla wszystkich zmiennych ilościowych z hipotez. 

In [ ]:
fig, axes = plt.subplots(len(quant_col),1,  figsize=(10,30))
for i, col in enumerate(quant_col):
    axes[i].set_title(f"Wykres pudełkowy kolumny {col}")
    sns.boxplot(data = data, x=data[col], ax=axes[i], orient = "h")

Wykresy pudełkowe pozwalają lepiej zapoznać sie z rozkładem zmiennych w zespole danych oraz dodatkowo pozwalają w łatwy sposób zauważyć wartości odstające w postaci punktów wystających poza wąsy (punkty te znajdują się w zakresach kwantyli 0-25 oraz 75-100). Jednak zawsze przed zakwalifikowaniem jakiś danych do wartości odstających należy się zastanowić, co powinno się z nimi zrobić i czy wpływają one w dużym stopniu negatywnie na modele predykcyjne. W naszym przypadku wykresy pudełkowe nie obrazują rozkładu zmiennych tak dokładnie, jak można było się tego spodziewać. Jest to spowodowane tym, że w każdym rekordzie danych, kolumna "Sales" ma inną wartość ilości sprzedanych gier. Poza wartościami 0 wszystkie można uznać jako odstające

Przeanalizujmy sprzedaż "NA_Sales"

In [ ]:
data[data["NA_Sales"]>=1]

Zastąpmy te 910 rekordów wartościami sprzedaży między 0 a 1 milionem sztuk

In [ ]:
import random
data.loc[data["NA_Sales"] >= 1,"NA_Sales"] = data["NA_Sales"].apply(lambda x: random.randrange(0,1))

Sprawdźmy jak zmienił się rozkład zmiennych

In [ ]:
sns.boxplot(data = data, x=data["NA_Sales"], orient = "h")

Podobny zabieg można byłoby wykonać na każdej zmiennej sprzedażowej "Sales".

Do wykonania wykresów pudełkowych skategoryzowanych wybrano pary: Zmienna jakościowa: Year, Zmienna ilościowa: JP_Sales Zmienna jakościowa: Genre, Zmianna ilościowa: EU_Sales

In [ ]:
sns.boxplot(data = data, x="Year", y = "JP_Sales", orient="v")

In [ ]:
sns.boxplot(data = data, x="Genre", y = "EU_Sales", orient="v")

Test normalny dla zmiennych - wartości odstające

Na samym początku sporządzono wykresy rozkładu wartości zmiennych ilościowych, aby sprawdzić czy dana analiza jest potrzebna i przydatna do analizy.

In [ ]:
fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quant_col):
    axes[i].set_title(f"Wykres rozkładu liczności dla {col}")
    sns.histplot(data[col], ax=axes[i], kde=True)

In [ ]:
for i, col in enumerate(quant_col):
    print(f"Wartość p dla testu normalnego dla kolumny {col} wynosi {normaltest(data[col].values).pvalue}\n")

Jak widać dla każdej kolumny zmiennej ilościowej nie posiadamy rozkładu normalnego. Jest to cecha tego zbioru danych i nie powinno się zmieniać jego wartości, aby rozkład zmiennych był bliski do rozkładu normalnego. Niestety z tego powodu nie możemy stosować parametrycznych metod statystycznych w celu przewidywania zmiennych zależnych.

# Indukcja drzew decyzyjnych

Do sprawdzenia hipotezy pierwszej dotyczącej udziału sprzedanych gier w Północnej Ameryce względem całego globu wybrano zmienne niezależne:

NA_Sales,
Global_Sales
Zmienne zależne wybrano dzięki obserwacjom z podpunktu drugiego. W tym wypadku mamy doczynienia z przewidywania wartości dlatego, należy wykorzystać drzewo regresyjne

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text, DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


# oddzielenie potrzebnych danych
hipo_1_data = data[["Global_Sales"]]
hipo_1_y = data["NA_Sales"]


# rozdzielenie danych na sety treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(hipo_1_data, hipo_1_y, train_size=0.7, random_state=1)

# ładowanie modelu
tree_model = DecisionTreeRegressor(min_samples_leaf=500)

# trenowanie modelu
tree_model.fit(X_train, y_train)

# sprawdzanie dokłądności modelu na danych testowych

preds = tree_model.predict(X_test)
r2score = r2_score(y_test, preds)
print(f"Współczynnik zbieżności wynosi {((1 - r2score) * 100):.2f}% - Dopasowanie modelu jest tym lepsze im bardziej współczynnik zbieżności jest bliżej 0%")

Określenie ważności predyktorów z użyciem wykresu.

In [ ]:
var_importances = tree_model.feature_importances_
std = np.std(var_importances,axis=0)
indices = np.argsort(var_importances)
plt.figure()
plt.title("Ważność predyktorów")
plt.barh(range(hipo_1_data.shape[1]), var_importances[indices],
       color="r", xerr=std, align="center")
plt.yticks(range(hipo_1_data.shape[1]), hipo_1_data.columns)
plt.ylim([-1, hipo_1_data.shape[1]])
plt.grid(b=True)
plt.xlim(0, 1)
plt.show()

Odczytanie i sformalizowanie na podstawie drzewa 3-5 reguł dla najbardziej wyrazistych klas lub dla liści o najmniejszej wariancji

In [ ]:
plt.figure(figsize=(25,20))
plot_tree(tree_model, filled=True, rounded=True, feature_names=X_train.columns)
plt.show()

Zamiast wariancji sklearn wykorzystuje MSE (Mean Square Error) do określenia jakości drzewa. Jak widać prawa część drzewa wygląda całkiem dobrze (MSE w liściach na niskim poziomie ok 0.08) co wskazuje na bardzo dobre dopasowanie średniej wartości w liściach do wartości rzeczywistych (ilości sprzedanych gier w wyrażonych w milionach). Lewa strona drzewa natomiast posiada bardzo niskie wartośći MSE (na poziomie około 0.01) co powoduje, że drzewo jest bardzo mało wiarygodne. Potwierdza to wartośc dopasowania modelu (47.33%) wskazując, że model jest dobrze dopasowany tylko w połowie. Dlatego hipotezę tą można przyjąć tylko dla gier, których nakład wyniósł powyżej 0.08 miliona sprzedanych sztuk. Dla całej populacji należy odrzucić hipotezę.

Do sprawdzenia hipotezy drugiej dotyczącej sprzedaży gier na rynek japoński w kolejnych latach wybrano następujące zmienne niezależne:

Year, JP_Sales, Publisher


In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix, precision_recall_fscore_support
from sklearn import preprocessing
pd.options.mode.chained_assignment = None

# oddzielenie potrzebnych danych
hipo_2_data = data[["Year", "Platform"]]
hipo_2_y = data["JP_Sales"]

decode = {
    "2600" : 1,
    "3DO" : 2,
    "3DS" : 3,
    "DC" : 4,
    "DS" : 5,
    "GB" : 6,
    "GBA" : 7,
    "GC" : 8,
    "GEN" : 9,
    "GG" : 10,
    "N64" : 11,
    "NES" : 12,
    "NG" : 13,
    "PC" : 14,
    "PCFX" : 15,
    "PS" : 16,
    "PS2" : 17,
    "PS3" : 18,
    "PS4" : 19,
    "PSP" : 20,
    "PSV" : 21,
    "SAT" : 22,
    "SCD" : 23,
    "SNES" : 24,
    "TG16" : 25,
    "WS" : 26,
    "Wii" : 27,
    "WiiU" : 28,
    "X360" : 29,
    "XB" : 30,
    "XOne" : 31
}

hipo_2_data.loc[:, "Platform"] = hipo_2_data.Platform.map(decode).values




In [ ]:
lab_enc = preprocessing.LabelEncoder()
hipo_2_y = lab_enc.fit_transform(hipo_2_y)

In [ ]:
# rozdzielenie danych na sety treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(hipo_2_data, hipo_2_y, train_size=0.7, random_state=1)
X_train

# ładowanie modelu
tree_model = DecisionTreeClassifier(min_samples_leaf=200)

# trenowanie modelu
tree_model.fit(X_train, y_train)

# sprawdzanie dokłądności modelu na danych testowych

preds = tree_model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Dokładność modelu wynosi {100*acc:.2f}%")

Określenie ważności predykatów z użyciem wykresu

In [ ]:
var_importances = tree_model.feature_importances_
std = np.std(var_importances,axis=0)
indices = np.argsort(var_importances)
plt.figure()
plt.title("Ważność predyktorów")
plt.barh(range(hipo_2_data.shape[1]), var_importances[indices],
       color="r", xerr=std, align="center")
plt.yticks(range(hipo_2_data.shape[1]), hipo_2_data.columns)
plt.ylim([-1, hipo_2_data.shape[1]])
plt.grid(b=True)
plt.xlim(0, 1)
plt.show()

In [ ]:
plt.figure(figsize=(25,20))
plot_tree(tree_model, filled=True, rounded=True, feature_names=X_train.columns)
plt.show()

Ze względu na ilość platform dostępnych w datasecie oraz ilość lat, w których gry były sprzedawane, powyższy wykres nie pozwala na odczytanie interesujących nas wartości. Po kolorach możemy tylko stwierdzić, że w prawej części drzewa jest dobre dopasowanie modelu. Jednak całościowo ma się to odpowiednio z obliczonym (63.45%) dokładnością modelu - drzewo też tylko w nieco większej połowie jest zakolorowane "dobrymi" kolorami.  

Do sprawdzenia hipotezy trzeciej dotyczącej gier akcji na rynku europejskim wybrano następujące zmienne niezależne:

Genre

In [ ]:

# oddzielenie potrzebnych danych
hipo_3_data = data[["Genre"]]
hipo_3_y = data["EU_Sales"]

decode = {
    "Action" : 1,
    "Adventure" : 2,
    "Fighting" : 3,
    "Misc" : 4,
    "Platform" : 5,
    "Puzzle" : 6,
    "Racing" : 7,
    "Role-Playing" : 8,
    "Shooter" : 9,
    "Simulation" : 10,
    "Sports" : 11,
    "Strategy" : 12
}

hipo_3_data.loc[:, "Genre"] = hipo_3_data.Genre.map(decode).values

In [ ]:
lab_enc = preprocessing.LabelEncoder()
hipo_3_y = lab_enc.fit_transform(hipo_3_y)

In [ ]:
# rozdzielenie danych na sety treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(hipo_3_data, hipo_3_y, train_size=0.7, random_state=1)

# ładowanie modelu
tree_model = DecisionTreeClassifier(min_samples_leaf=500)

# trenowanie modelu
tree_model.fit(X_train, y_train)

# sprawdzanie dokłądności modelu na danych testowych

preds = tree_model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Dokładność modelu wynosi {100*acc:.2f}%")

Określenie ważności predykatów

In [ ]:
var_importances = tree_model.feature_importances_
std = np.std(var_importances,axis=0)
indices = np.argsort(var_importances)
plt.figure()
plt.title("Ważność predyktorów")
plt.barh(range(hipo_3_data.shape[1]), var_importances[indices],
       color="r", xerr=std, align="center")
plt.yticks(range(hipo_3_data.shape[1]), hipo_3_data.columns)
plt.ylim([-1, hipo_3_data.shape[1]])
plt.grid(b=True)
plt.xlim(0, 1)
plt.show()

In [ ]:
plt.figure(figsize=(25,20))
plot_tree(tree_model, filled=True, rounded=True, feature_names=X_train.columns, class_names=["1","2", "3", "4", "5", "6","7", "8", "9", "10", "11", "12"])
plt.show()

# Analiza skupień

Do wykonania analizy skupień wybrano zmienne:

* Genre
* Year
* Global_Sales

Celem analizy jest sprawdzenie, czy jesteśmy w stanie wyznaczyć jakieś konkretne grupy gier kupowanych przez konsumentów w zależności od roku.

In [ ]:
from sklearn.cluster import KMeans
pd.options.mode.chained_assignment = None
from sklearn.decomposition import PCA

decode = {
    "Action" : 1,
    "Adventure" : 2,
    "Fighting" : 3,
    "Misc" : 4,
    "Platform" : 5,
    "Puzzle" : 6,
    "Racing" : 7,
    "Role-Playing" : 8,
    "Shooter" : 9,
    "Simulation" : 10,
    "Sports" : 11,
    "Strategy" : 12
}

# oddzielenie potrzebnych danych
cluster_data = data[["Genre", "Year", "Global_Sales"]]
cluster_data.loc[:, "Genre"] = cluster_data.Genre.map(decode).values

sse = []

# ładowanie modelu
for k in range(1,11):
    cluster_model = KMeans(n_clusters=k)
    cluster_model.fit(cluster_data)
    sse.append(cluster_model.inertia_)
    
# sprawdzanie jaką wartość k wybrać - metoda łokcia

plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Wartość k")
plt.ylabel("SSE")
plt.show()

Na powyższym wykresie  nie jest oczywista najrozsądniejsza ilość klastrów jaką powinno się wybrać. Podejrzewamy wartość 2 lub 3 lecz do automatycznego doboru ilości klastrów można wykorzystać silhouette coefficient score.

Silhouette Coefficient jest obliczany przy użyciu średniej odległości wewnątrz klastra (a) i średniej odległości do najbliższego klastra (b) dla każdej próbki. Współczynnik sylwetki dla próbki to (b - a) / max (a, b). Aby wyjaśnić, b to odległość między próbką a najbliższą gromadą, której próbka nie jest częścią. Zwróć uwagę, że silhouette coefficient jest definiowany tylko wtedy, gdy liczba etykiet wynosi 2 <= n_labels <= n_samples -

In [ ]:
from sklearn.metrics import silhouette_score

sil_score_max = -1
best_n_clusters = 0

for k in range(2,11):
  cluster_model = KMeans(n_clusters = k)
  labels = cluster_model.fit_predict(cluster_data)
  sil_score = silhouette_score(cluster_data, labels)
  print(f"Średnia wartość silhouette score dla {k} klastrów wynosi {sil_score}")
  if sil_score > sil_score_max:
    sil_score_max = sil_score
    best_n_clusters = k
    
print(f"Najlepsza ilość klastrów to: {best_n_clusters}")

In [ ]:
cluster_model = KMeans(n_clusters = best_n_clusters)
# trenowanie modelu
result = cluster_model.fit_predict(cluster_data)

labels = cluster_model.labels_

result_data = cluster_data.copy()
result_data["labels"] = labels

results_0 = cluster_data[result_data.labels == 0]
results_1 = cluster_data[result_data.labels == 1]

Analiza klastra 1:

In [ ]:
results_0.describe()

Analiza klastra 2:

In [ ]:
results_1.describe()

Z powodu jednolitego rozkładu zmiennych badanie skupień nie przynosi dodatkowych informacji, ponieważ metody typu najniższych sąsiadów dzielą te zbiory na podobne klatry o podobnym wyglądzie. Jedyna różnica pomiędzy klastrami jest w sprzedaży globalnej, gdzie pierwszy klaster skupia w sobie gry, których sprzedano więcej, a drugi klaster te gry, których sprzedano mniej. Patrząc na typ gry, bądź rok publikacji, to nie ma ona większego wpływu na grupy gier.

Analiza skupień metodą EM

In [ ]:
# załadowanie modułu do analizy metodą EM
from sklearn.mixture import GaussianMixture

# wczytanie modelu z dwoma klastrami
em_model = GaussianMixture(n_components=2, random_state=0)

labels = em_model.fit_predict(cluster_data)

#labels = em_model.labels_

result_data = cluster_data.copy()
result_data["labels"] = labels

results_0 = cluster_data[result_data.labels == 0]
results_1 = cluster_data[result_data.labels == 1]

Analiza klastra 1

In [ ]:
results_0.describe()

Analiza klastra 2

In [ ]:
results_1.describe()

Porównując wyniki pomiędzy analizą skupień metodą K-najbliższych sąsiadów a metodą EM, można zauważyć niewielkie różnice. Analiza metodą EM podzieliła zbiór danych głównie ze względu na globalną sprzedaż (dla klastra 1 gry sprzedane między 0.01 a 0.73 miliona sztuk, a dla klastra 2 od 0.02 do 82.74 miliona sztuk), ale w odróżnieniu od K-najbliższych sąsiadów wzięła również pod uwagę rok publikacji, gdzie klaster 2 jest nieco bardziej przesunięty w prawą stronę i średnia wartość wynosi dla niego 2004 rok sprzedaży, natomiast dla klastra 1 wynosi 2006 rok.

Wykres dla każdej zmiennej

In [ ]:
import scipy.stats as stats
import math

for col in results_1:
    mu_0 = results_0[col].mean()
    variance_0 = results_0[col].var()
    mu_1 = results_1[col].mean()
    variance_1 = results_1[col].var()
    sigma_0 = math.sqrt(variance_0)
    sigma_1 = math.sqrt(variance_1)
    x_0 = np.linspace(mu_0 - 3*sigma_0, mu_0 + 3*sigma_0, 100)
    x_1 = np.linspace(mu_1 - 3*sigma_1, mu_1 + 3*sigma_1, 100)
    plt.figure(figsize=(8,5))
    plt.title(f"Wykres dystrybucji zmiennych w klastrach dla zmiennej {col}")
    plt.plot(x_0, stats.norm.pdf(x_0, mu_0, sigma_0))
    plt.plot(x_1, stats.norm.pdf(x_1, mu_1, sigma_1))
    plt.show()

# Algorytm Data Mining

Jako dodatkowy algorytm wybrano sieci neuronowe implementowane przy pomocy biblioteki tensorflow. Model ten został wybrany, ponieważ sieci neuronowe są szeroko wykorzystywane w data science.

In [ ]:
# zapisywanie danych w postaci tensora
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

hipo_1_data = data[["Year", "Genre"]]
hipo_1_y = data["Global_Sales"]

decode = {
    "Action" : 1,
    "Adventure" : 2,
    "Fighting" : 3,
    "Misc" : 4,
    "Platform" : 5,
    "Puzzle" : 6,
    "Racing" : 7,
    "Role-Playing" : 8,
    "Shooter" : 9,
    "Simulation" : 10,
    "Sports" : 11,
    "Strategy" : 12
}

hipo_1_data.loc[:, "Genre"] = hipo_1_data.Genre.map(decode).values

# rozdzielenie danych na sety treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(hipo_1_data, hipo_1_y, train_size=0.7, random_state=1)

model = keras.models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(10, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(10, activation="relu"),
    layers.Dense(100, activation="softmax")
])

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

model.build()
model.summary()

Model jaki zbudowałem składa się z 5 warstw:

* Warstwa wejściowa - składa się z 3 neuronów (po jednym na każdą zmienną)
* Pierwsza warstwa wewnętrzna - składa się z 10 neuronów, każdy połączony z neuronami z warstwy wejściowej. Zastosowano w nim funkcję aktywacyjną relu. Neuron z funkcją aktywacji ReLU przyjmuje dowolne wartości rzeczywiste jako swoje wejście (a), ale aktywuje się tylko wtedy, gdy te wejście (a) są większe niż 0.
* Druga warstwa wewnętrzna - warstwa przejściowa, która losowo ustawia jednostki wejściowe na 0 z częstotliwością na każdym kroku podczas treningu, co pomaga zapobiegać nadmiernemu dopasowaniu. Wejścia nie ustawione na 0 są skalowane w górę o 1 / (1 - stawka) tak, że suma wszystkich wejść pozostaje niezmieniona.
* Trzecia warstwa wewnętrzna - składa się z 10 neuronów, który każdy połączony jest z neuronem drugiej warstwy wewnętrznej. W warstwie tej zastosowaną funkcję aktywacyjną relu.
* Warstwa wyjściowa - składa się z dwóch neuronów, który każdy jest połączony z trzecią warstwą wewnętrzną. Zastosowano tutaj funkcję aktywacyjną softmax, która przetwarza wartości na końcu neuronów i zamienia je na wartości między 0 a 1

Dodatkowo sieć została zbudowana z określeniem optymalizatora Adam, który implementuje wykładniczą średnią ruchomą gradientów, aby skalować tempo uczenia. Utrzymuje wykładniczo malejącą średnią poprzednich gradientów. Adam jest wydajny obliczeniowo i ma bardzo małe wymagania dotyczące pamięci. Adam Optimizer jest jednym z najpopularniejszych algorytmów optymalizacji zstępowania gradientu. Oprócz optymalizatora jako funkcję kosztów wybrano sparse_categorical_crossentropy, a do pomiarów jakości sieci wyprano parametr accuracy.

In [ ]:
#trenowanie modelu
history = model.fit(X_train, y_train, validation_split=0.33, epochs=20)
# summarize history for accuracy
plt.figure(figsize=(8,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Dokładność modelu')
plt.ylabel('Dokładność')
plt.xlabel('Epoka')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Funkcja kosztów modelu')
plt.ylabel('Strata')
plt.xlabel('Epoka')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Standaryzacja cech

In [ ]:
from sklearn import preprocessing

In [ ]:
scaler = preprocessing.StandardScaler()
X_train_scal = scaler.fit_transform(X_train)
X_test_scal = scaler.transform(X_test)

#trenowanie modelu
history = model.fit(X_train, y_train, validation_split=0.33, epochs=20)
# summarize history for accuracy
plt.figure(figsize=(8,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Dokładność modelu')
plt.ylabel('Dokładność')
plt.xlabel('Epoka')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Funkcja kosztów modelu')
plt.ylabel('Strata')
plt.xlabel('Epoka')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

Porównując modele drzewa decyzyjnego i sieci neuronowej w celu potwierdzenia hipotezy 1 model drzewa sprawdzał się dużo lepiej niż sieć neuronowa nie (różnica o 10%). Oznacza to, że nie zawsze skomplikowane rozwiązania są dobre dla każdego rozwiązania. W sytuacji, kiedy jest niewielka korelacja zmiennych, a rozkłady nie są normalne modele nieparametryczne mogą sprawdzać się lepiej niż modele parametryczne. Jednakże hipotezę można potwierdzić przy wykorzystaniu modelu sieci neuronowej